In [7]:
#Lib
import pandas as pd
from joblib import load
#Clustering
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
#Zips
import py7zr
#elbow point
%pip install kneed
from kneed import KneeLocator
from sklearn.preprocessing import MinMaxScaler
# plots 
%pip install plotly
import plotly.express as px
import matplotlib.pyplot as plt
#Save plots as .png
%pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
#Defining target column
target = 'TARGET'
target_value = 1.0
name = 'telecom_churn_me'

In [9]:
#Accessing transformed data
with py7zr.SevenZipFile(f'../../../../data/{name}/fully_transformed.7z', mode='r') as z:
    z.extractall(path='fully_transformed/')

In [10]:
#Reading transformed data
df = pd.read_csv('./fully_transformed/fully_transformed.csv')
df

,PARTY_NATIONALITY,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed,TARGET
0,-0.557334,0.246921,-0.474652,0.070284,0.322559,0.012830,-0.220822,0.158250,0.252462,0.207486,-0.073290,-0.446350,-0.192129,0.034223,2.036845,0
1,2.044388,-0.487195,-0.474652,-0.091379,-0.372874,-0.114327,-0.220822,-0.317304,-0.154569,-0.167244,-0.154028,1.999755,-0.164464,-0.309134,0.071011,0
2,-0.410067,0.132757,0.893471,-0.059725,-0.372874,0.012830,-0.220822,0.019869,-0.192753,-0.044365,0.360690,-0.300696,-0.191567,-0.274824,-0.584267,0
3,-0.557334,-0.483142,-0.474652,-0.067556,-0.372874,0.012830,-0.220822,-0.112246,-0.432914,-0.384486,-0.301507,-0.418739,-0.192129,-0.139412,-0.748087,0
4,-0.557334,0.486317,-0.474652,0.080422,0.496418,0.139987,-0.220822,-0.293407,-0.403410,-0.189150,-0.288009,-0.446350,-0.192129,-0.024835,0.071011,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984879,-0.410067,-0.430476,-0.474652,-0.091525,-0.025157,-0.114327,-0.220822,0.014112,1.304946,-0.343915,0.874575,-0.377061,0.265396,-0.180548,-0.256628,0
984880,-0.557334,-0.571080,0.893471,-0.067483,-0.199016,-0.114327,-0.220822,-0.436092,-0.427813,-0.445148,-0.301507,-0.446350,-0.192129,-0.100294,-0.420448,0
984881,-0.557334,0.840817,0.893471,0.039322,-0.199016,0.012830,1.868853,-0.361242,-0.432914,-0.408854,-0.301507,-0.399517,-0.192129,-0.108054,-0.748087,0
984882,-0.410067,-0.540825,-0.474652,-0.063311,-0.199016,0.012830,-0.220822,0.092889,-0.263887,-0.418057,0.116767,-0.361159,-0.179209,-0.274563,-0.584267,0


In [11]:
#Preparing dataset for clustering
y = df[target]
x = df.drop([target], axis=1)
x

,PARTY_NATIONALITY,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed
0,-0.557334,0.246921,-0.474652,0.070284,0.322559,0.012830,-0.220822,0.158250,0.252462,0.207486,-0.073290,-0.446350,-0.192129,0.034223,2.036845
1,2.044388,-0.487195,-0.474652,-0.091379,-0.372874,-0.114327,-0.220822,-0.317304,-0.154569,-0.167244,-0.154028,1.999755,-0.164464,-0.309134,0.071011
2,-0.410067,0.132757,0.893471,-0.059725,-0.372874,0.012830,-0.220822,0.019869,-0.192753,-0.044365,0.360690,-0.300696,-0.191567,-0.274824,-0.584267
3,-0.557334,-0.483142,-0.474652,-0.067556,-0.372874,0.012830,-0.220822,-0.112246,-0.432914,-0.384486,-0.301507,-0.418739,-0.192129,-0.139412,-0.748087
4,-0.557334,0.486317,-0.474652,0.080422,0.496418,0.139987,-0.220822,-0.293407,-0.403410,-0.189150,-0.288009,-0.446350,-0.192129,-0.024835,0.071011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984879,-0.410067,-0.430476,-0.474652,-0.091525,-0.025157,-0.114327,-0.220822,0.014112,1.304946,-0.343915,0.874575,-0.377061,0.265396,-0.180548,-0.256628
984880,-0.557334,-0.571080,0.893471,-0.067483,-0.199016,-0.114327,-0.220822,-0.436092,-0.427813,-0.445148,-0.301507,-0.446350,-0.192129,-0.100294,-0.420448
984881,-0.557334,0.840817,0.893471,0.039322,-0.199016,0.012830,1.868853,-0.361242,-0.432914,-0.408854,-0.301507,-0.399517,-0.192129,-0.108054,-0.748087
984882,-0.410067,-0.540825,-0.474652,-0.063311,-0.199016,0.012830,-0.220822,0.092889,-0.263887,-0.418057,0.116767,-0.361159,-0.179209,-0.274563,-0.584267


In [12]:
#Getting number of k-means clusters
def elbow(dataset):
    scaler = MinMaxScaler()
    scaler.fit(dataset)
    X=scaler.transform(dataset)
    inertia = []
    max_clusters = dataset.shape[1] + 1 #number of variables
    cluster_number = list(range(1, max_clusters))
    for i in cluster_number:
        kmeans = KMeans(
            n_clusters=i, init="k-means++",
            n_init=10,
            tol=1e-04, random_state=42
        )
        kmeans.fit(X)
        inertia.append(kmeans.inertia_)
    #locating elbow point
    kneedle = KneeLocator(cluster_number, inertia, S=1.0, curve="convex", direction="decreasing")
    clusters_number = kneedle.knee
    if (cluster_number != int):
        kneedle = KneeLocator(cluster_number, inertia, S=0.0, curve="convex", direction="decreasing")
        clusters_number = kneedle.knee
    return clusters_number

In [13]:
clusters_number = elbow(x)
clusters_number

5

In [14]:
#Clustering function
def clustering(dataframe, clusters_number):
    dataframe.reset_index
    #k-means algorithm
    kmeans = KMeans(
            n_clusters=clusters_number, #number of clusters for general dataset
            init="k-means++",
            n_init=10,
            tol=1e-04, 
            random_state=42
        )
    #fitting the algorithm
    kmeans.fit(dataframe)
    #labeling
    clusters=pd.DataFrame(dataframe)
    clusters['label']=kmeans.labels_
    return clusters

In [15]:
#Transformed dataframe labeled 
df = pd.DataFrame(clustering(x, clusters_number))
#Main clustering polar graph
polar=df.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig0 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig0.show()
fig0.write_image(f'main_cluster_img.png')
#Main clustering pie plot to see clustering distribution
pie0=df.groupby('label').size().reset_index()
pie0.columns=['label','value']
pie0 = px.pie(pie0,values='value',names='label')
pie0.show()
pie0.write_image(f'main_cluster_distribution.png')


/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trace_data = trace_data.append(trace_data.iloc[0])
/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trace_data = trace_data.append(trace_data.iloc[0])
/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trace_data = trace_data.append(trace_data.iloc[0])
/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trace_data = trace_

In [16]:
df.to_csv(f'./main_cluster.csv') #creating filtered cvs 
#Main dataframe clustered
df

,PARTY_NATIONALITY,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed,label
0,-0.557334,0.246921,-0.474652,0.070284,0.322559,0.012830,-0.220822,0.158250,0.252462,0.207486,-0.073290,-0.446350,-0.192129,0.034223,2.036845,3
1,2.044388,-0.487195,-0.474652,-0.091379,-0.372874,-0.114327,-0.220822,-0.317304,-0.154569,-0.167244,-0.154028,1.999755,-0.164464,-0.309134,0.071011,1
2,-0.410067,0.132757,0.893471,-0.059725,-0.372874,0.012830,-0.220822,0.019869,-0.192753,-0.044365,0.360690,-0.300696,-0.191567,-0.274824,-0.584267,4
3,-0.557334,-0.483142,-0.474652,-0.067556,-0.372874,0.012830,-0.220822,-0.112246,-0.432914,-0.384486,-0.301507,-0.418739,-0.192129,-0.139412,-0.748087,0
4,-0.557334,0.486317,-0.474652,0.080422,0.496418,0.139987,-0.220822,-0.293407,-0.403410,-0.189150,-0.288009,-0.446350,-0.192129,-0.024835,0.071011,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984879,-0.410067,-0.430476,-0.474652,-0.091525,-0.025157,-0.114327,-0.220822,0.014112,1.304946,-0.343915,0.874575,-0.377061,0.265396,-0.180548,-0.256628,2
984880,-0.557334,-0.571080,0.893471,-0.067483,-0.199016,-0.114327,-0.220822,-0.436092,-0.427813,-0.445148,-0.301507,-0.446350,-0.192129,-0.100294,-0.420448,4
984881,-0.557334,0.840817,0.893471,0.039322,-0.199016,0.012830,1.868853,-0.361242,-0.432914,-0.408854,-0.301507,-0.399517,-0.192129,-0.108054,-0.748087,4
984882,-0.410067,-0.540825,-0.474652,-0.063311,-0.199016,0.012830,-0.220822,0.092889,-0.263887,-0.418057,0.116767,-0.361159,-0.179209,-0.274563,-0.584267,0


In [17]:
#Create a csv focusing in cluster churns
clust = 0 #iterator
df[target] = y
while clust < clusters_number:
    dataframe = pd.DataFrame(df.loc[df['label'] == clust]) #Filter by cluster
    dataframe = dataframe.loc[dataframe[target] == target_value] #Filter by positive targets
    dataframe = dataframe.drop(columns=[target,'label'])
    dataframe.to_csv(f'./cluster{clust}.csv') #creating filtered cvs 
    clust = clust + 1 #iterator
dataframe

,PARTY_NATIONALITY,BILL_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,OTHER_LINES,STATUS,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER,MOUS_FROM_INT_NUMBER,DATA_IN_BNDL,Years_stayed
24,-0.557334,-0.504839,0.893471,-0.027196,-0.025157,-0.114327,-0.220822,-0.436092,-0.432914,-0.445148,-0.301507,-0.446350,-0.192129,0.272471,-0.584267
32,-0.410067,-0.131177,0.893471,-0.037549,1.191851,0.012830,-0.220822,-0.436092,-0.432914,-0.445148,-0.301507,-0.446350,-0.192129,-0.312506,-0.748087
164,-0.557334,-0.006125,0.893471,-0.081564,-0.199016,-0.114327,-0.220822,0.299671,-0.356679,-0.064844,-0.225684,-0.446350,-0.192129,0.285478,-0.584267
200,0.718982,-0.092633,0.893471,-0.083025,0.322559,-0.114327,-0.220822,0.067284,-0.432914,-0.389412,-0.301507,-0.446350,-0.192129,-0.171457,-0.748087
202,-0.410067,-0.642839,0.893471,-0.076115,-0.199016,0.012830,-0.220822,-0.003717,0.432345,-0.195760,0.002950,-0.433986,-0.183001,-0.311201,-0.911906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983817,-0.557334,0.092246,0.893471,-0.068514,-0.199016,-0.114327,-0.220822,1.947860,-0.375211,0.089791,-0.246556,-0.446350,-0.192129,0.448903,-0.911906
984086,-0.557334,0.101081,2.261594,0.025976,0.322559,0.012830,-0.220822,0.089129,-0.432914,-0.229850,-0.301507,-0.410657,-0.192129,-0.203181,-0.256628
984191,-0.410067,0.297072,0.893471,-0.078785,-0.025157,-0.114327,-0.220822,-0.123310,-0.432914,-0.370228,-0.301507,-0.446350,-0.192129,-0.077578,-0.748087
984645,-0.017354,-0.530250,0.893471,-0.084965,-0.372874,0.012830,1.868853,-0.436092,-0.432914,-0.445148,-0.301507,-0.446350,-0.192129,-0.312173,-0.584267


In [18]:
clust = 0 #iterator
while clust < clusters_number:
    sub_cluster = pd.read_csv(f'./cluster{clust}.csv') #read every subcluster .csv
    sub_cluster = sub_cluster.drop(columns=['Unnamed: 0']) #drop the added column
    sub_clusters_number = elbow(sub_cluster) #calculate elbow point for every sub-cluster
    clusters = pd.DataFrame(clustering(sub_cluster, sub_clusters_number)) #k-means clustering method
    #polar sub-clusters graph
    sub_polar=clusters.groupby("label").mean().reset_index() 
    sub_polar=pd.melt(sub_polar,id_vars=["label"])
    fig = px.line_polar(sub_polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
    fig.show() #print here
    fig.write_image(f'cluster{clust}img.png') #save as .png file
    #Pie plot to see sub-cluster's distribution
    pie=clusters.groupby('label').size().reset_index()
    pie.columns=['label','value']
    pie = px.pie(pie,values='value',names='label')
    pie.show() #print here
    pie.write_image(f'cluster{clust}distribution.png') #save as .png file
    clust = clust + 1
    
    

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future ve

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future ve

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future ve

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

